In [1]:
import json
import logging

import torch
from tqdm import tqdm

from torch.utils.data import DataLoader, SequentialSampler

In [2]:
import sys
sys.path.insert(0, '..')

In [3]:
from transformers import (
    WEIGHTS_NAME,
    GPT2Config,
    GPT2LMHeadModel,
    GPT2Tokenizer,
)

In [4]:
try:
    from torch.utils.tensorboard import SummaryWriter
except ImportError:
    from tensorboardX import SummaryWriter

logger = logging.getLogger(__name__)

In [5]:
MODEL_PATH = '../output/gpt2_base/'
MAX_LEN = 20
STOP_TOKEN = '<|endoftext|>'

In [6]:
MODEL_CLASSES = {
    "gpt2": (GPT2Config, GPT2LMHeadModel, GPT2Tokenizer)
}
config = GPT2Config.from_pretrained(MODEL_PATH)
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')


In [7]:
model = GPT2LMHeadModel.from_pretrained(MODEL_PATH)

In [8]:
model = model.cuda()
model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 1024)
    (wpe): Embedding(1024, 1024)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2):

In [19]:
def get_predictions(input_sentence):
    encoded_prompt = tokenizer.encode(input_sentence, add_special_tokens=False, return_tensors='pt')
    encoded_prompt = encoded_prompt.cuda()
#     print(f"Encoded prompt : {encoded_prompt}")
    output_dict = {'answer': ''}
    with torch.no_grad():

        out = model.generate(
            # input_ids: `torch.LongTensor` of shape `(batch_size, sequence_length)`
            input_ids=encoded_prompt,
            max_length= MAX_LEN + encoded_prompt.size(-1),
            temperature=1.0,
            top_k=0,
            top_p=0.9,
            do_sample=True,
            num_return_sequences=1,
        )
        
        for out_seq in out:
            text = tokenizer.decode(out_seq, clean_up_tokenization_spaces=True)
            text = text[: text.find(STOP_TOKEN) if STOP_TOKEN else None]
            answer = text[text.find('?')+1:]
            output_dict['answer'] += answer
            output_dict['answer'] += '. '
            
        return output_dict

In [10]:
model.cuda()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 1024)
    (wpe): Embedding(1024, 1024)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2):

In [15]:
get_predictions('And then, Brighten up a worn stainless steel countertop surface and/or trim.. Now, what happens to brightness of counter top?')

Encoded prompt : tensor([[ 1870,   788,    11, 17558,   268,   510,   257, 12666, 25704,  7771,
          3753,  4852,  4417,   290,    14,   273, 15797,   492,  2735,    11,
           644,  4325,   284, 22204,   286,  3753,  1353,    30]],
       device='cuda:0')


{'answer': 'counter top was dull before, and brightened afterwards.. '}

In [16]:
test_input = []
with open('../data/trackworld/simplified/ea_positive/test.jsonl', 'r') as open_file:
    for line in open_file:
        test_input.append(json.loads(line))

In [20]:
with open('../output/gpt2_base/predictions/dev/output.jsonl', 'w') as open_file:
    for item in test_input:
        output = get_predictions(item['question'])
        output['id'] = item['id']
        json.dump(output, open_file)
        open_file.write('\n')

In [26]:
from typing import List, Dict
import itertools

def read_jsonl(input_file: str) -> List[Dict]:
    output: List[Dict] = []
    with open(input_file, 'r') as open_file:
        for line in open_file:
            output.append(json.loads(line))
    return output

def aggregate_predictions(prediction_fp: str, out_fp: str, separator="||"):
    json_lines = read_jsonl(prediction_fp)
    # example of an id is: "wikihow.com/...||1||1||2"
    #   might indicate wikihow article's sentence 1, entity 1, attribute 2
    # Aggregate by sentence (e.g., "wikihow.com/...||1) to get an array of state changes
    with open(out_fp, 'w') as outfile:
        for grp_id, grp in itertools.groupby(json_lines, lambda line:separator.join(line["id"].split(separator)[0:2])):
            fmt_j = {"id": grp_id, "answers": []}
            for line in grp:
                # This is in the format required by the evaluator.
                fmt_j["answers"].append(line["answer"])
            print(json.dumps(fmt_j), file=outfile)

In [27]:
aggregate_predictions('../output/gpt2_base/predictions/dev/output.jsonl','../output/gpt2_base/predictions/dev/output_aggregated.jsonl' )